In [50]:
import requests
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Base URL for the Berkeley PD Cases API
base_url = "https://services7.arcgis.com/vIHhVXjE1ToSg0Fz/arcgis/rest/services/Berkeley_PD_Cases_2016_to_Current/FeatureServer/0/query"

# Use the DATE literal to get cases that occurred on April 7, 2025
where_clause = (
    "Occurred_Datetime >= DATE '2025-04-06 00:00:00' "
    "AND Occurred_Datetime < DATE '2025-04-08 00:00:00'"
)

# Set the fields you want to retrieve
params = {
    "where": where_clause,
    "outFields": "Incident_Type,Statute_Type,Block_Address,Occurred_Datetime,Statute_Description",
    "outSR": "4326",
    "f": "json"
}

# Make the GET request
response = requests.get(base_url, params=params)
response.raise_for_status()

# Parse the JSON response
data = response.json()
features = data.get("features", [])
records = [feature.get("attributes", {}) for feature in features]

# Convert to DataFrame
df = pd.DataFrame(records)

# Convert Occurred_Datetime from epoch ms to readable datetime
if "Occurred_Datetime" in df.columns:
    df["Occurred_Datetime"] = pd.to_datetime(df["Occurred_Datetime"], unit="ms")

# Show results
if df.empty:
    print("No cases found for April 7, 2025.")
else:
    print("First 5 Berkeley PD cases for April 7, 2025:")
    print(df.head())
    print(f"\nTotal cases found: {len(df)}")


First 5 Berkeley PD cases for April 7, 2025:
                    Incident_Type       Statute_Type  \
0             Narcotics Violation  Health and Safety   
1             Narcotics Violation  Health and Safety   
2  Theft Misdemeanor (Under $950)         Penal Code   
3             Narcotics Violation  Health and Safety   
4             Narcotics Violation         Penal Code   

                Block_Address   Occurred_Datetime  \
0  ADELINE ST  &  FAIRVIEW ST 2025-04-06 05:41:00   
1  ADELINE ST  &  FAIRVIEW ST 2025-04-06 05:41:00   
2         1400 UNIVERSITY AVE 2025-04-07 02:59:00   
3      8TH ST  &  HARRISON ST 2025-04-07 22:51:00   
4      8TH ST  &  HARRISON ST 2025-04-07 22:51:00   

                    Statute_Description  
0          POSSESS CONTROLLED SUBSTANCE  
1        POSSESS UNLAWFUL PARAPHERNALIA  
2          SHOPLIFTING (LESS THAN $950)  
3        POSSESS UNLAWFUL PARAPHERNALIA  
4  PROBATION VIOLATION: REARREST/REVOKE  

Total cases found: 54


In [48]:
# Step 1: Aggregate the data to get a count per street.
street_counts = df.groupby('Street').size().reset_index(name='Crime_Count')
print(street_counts)

                                          Street  Crime_Count
0                                 1000 GILMAN ST            2
1                            1000 UNIVERSITY AVE            3
2                                1200 DWIGHT WAY            2
3                            1200 UNIVERSITY AVE            2
4                              1400 SHATTUCK AVE            1
5                            1400 UNIVERSITY AVE            1
6                             1600 SAN PABLO AVE            3
7                                  1900 HASTE ST            1
8                             1900 SAN PABLO AVE            1
9                              2000 CHANNING WAY            5
10                             2000 SHATTUCK AVE            1
11                                   2100 4TH ST            1
12                                   2100 5TH ST            1
13                                2100 CENTER ST            1
14                2100 MARTIN LUTHER KING JR WAY            3
15      

In [49]:
# Step 2: Geocode the street addresses.
geolocator = Nominatim(user_agent="crime_map_app")
# Using rate limiter to avoid overloading the service.
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_lat_long(address):
    try:f
        location = geocode(address)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    return None, None

# Get coordinates for each unique street
latitudes = []
longitudes = []
for street in street_counts['Street']:
    lat, lon = get_lat_long(street)
    latitudes.append(lat)
    longitudes.append(lon)
    # Optional: Sleep a bit if running many addresses; RateLimiter is usually sufficient.
    time.sleep(1)
    
street_counts['Latitude'] = latitudes
street_counts['Longitude'] = longitudes

# Drop any entries that could not be geocoded.
street_counts = street_counts.dropna(subset=['Latitude', 'Longitude'])

print(street_counts)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('2100 4TH ST',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py"

                                          Street  Crime_Count   Latitude  \
0                                 1000 GILMAN ST            2  37.879926   
1                            1000 UNIVERSITY AVE            3  37.868489   
2                                1200 DWIGHT WAY            2  37.861161   
3                            1200 UNIVERSITY AVE            2  40.838112   
4                              1400 SHATTUCK AVE            1  37.881814   
5                            1400 UNIVERSITY AVE            1  43.074433   
6                             1600 SAN PABLO AVE            3  37.806489   
7                                  1900 HASTE ST            1  37.864226   
8                             1900 SAN PABLO AVE            1  37.809169   
9                              2000 CHANNING WAY            5  37.865354   
10                             2000 SHATTUCK AVE            1  37.871883   
14                2100 MARTIN LUTHER KING JR WAY            3  37.870176   
15          

In [53]:
# Step 3: Define a function to map crime count to a color.
def get_color(crime_count):
    """
    Define thresholds for crime levels.
    Adjust thresholds as needed.
    """
    if crime_count >= 5:         # High crime count threshold (example)
        return 'red'
    elif crime_count >= 3:       # Moderate crime count threshold (example)
        return 'orange'
    else:
        return 'green'

In [54]:
# Step 4: Create a map centered on a rough central location.
# You may set the initial latitude and longitude based on your city, here we use the first available point as center.
if not street_counts.empty:
    center_lat = street_counts.iloc[0]['Latitude']
    center_lon = street_counts.iloc[0]['Longitude']
else:
    center_lat, center_lon = 0, 0

crime_map = folium.Map(location=[center_lat, center_lon], zoom_start=13)

In [55]:
# Step 5: Add markers for each street.
for idx, row in street_counts.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=8,
        popup=f"{row['Street']}: {row['Crime_Count']} crimes",
        color=get_color(row['Crime_Count']),
        fill=True,
        fill_color=get_color(row['Crime_Count']),
        fill_opacity=0.7
    ).add_to(crime_map)

# Save the map to an HTML file and display it in a browser if needed.
crime_map.save("crime_map.html")
print("Map has been saved to crime_map.html")

Map has been saved to crime_map.html


In [58]:
import pandas as pd
import folium
import requests
import time
from shapely.geometry import LineString

# Helper: Extract streets from intersections
def extract_street_names(address):
    return [s.strip() for s in address.split('&')]

df['Streets'] = df['Block_Address'].apply(extract_street_names)
flat_streets = df.explode('Streets')

# Count crimes per individual street
street_counts = flat_streets.groupby('Streets').size().reset_index(name='Crime_Count')

# Color mapping function
def get_color(crime_count):
    if crime_count >= 5:
        return 'red'
    elif crime_count >= 3:
        return 'orange'
    else:
        return 'green'

# Create the map centered around Berkeley
berkeley_center = [37.8715, -122.2730]
crime_map = folium.Map(location=berkeley_center, zoom_start=14)

# Feature group for zoom-level markers
marker_group = folium.FeatureGroup(name='Crime Markers (Zoom In)')

# Overpass query helper
def get_street_geometry(street_name, city='Berkeley', state='California'):
    query = f"""
    [out:json];
    area["name"="{city}"]["is_in:state"="{state}"]->.searchArea;
    way["highway"]["name"="{street_name}"](area.searchArea);
    out geom;
    """
    try:
        response = requests.get("https://overpass-api.de/api/interpreter", params={'data': query})
        data = response.json()
        return data['elements']
    except Exception as e:
        print(f"Overpass error for {street_name}: {e}")
        return []

# Go through each street
for _, row in street_counts.iterrows():
    street_name = row['Streets']
    crime_count = row['Crime_Count']
    color = get_color(crime_count)
    
    # Draw full street lines
    elements = get_street_geometry(street_name)
    for el in elements:
        if el['type'] == 'way' and 'geometry' in el:
            coords = [(pt['lat'], pt['lon']) for pt in el['geometry']]
            folium.PolyLine(locations=coords, color=color, weight=5, opacity=0.7).add_to(crime_map)

    # Add markers for zoom-level
    # We'll approximate lat/lon as the midpoint of the street's segments (for demo)
    if elements:
        coords = [(pt['lat'], pt['lon']) for pt in elements[0].get('geometry', [])]
        if coords:
            mid_index = len(coords) // 2
            lat, lon = coords[mid_index]
            marker_group.add_child(folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color=color,
                fill=True,
                fill_color=color,
                popup=f"{street_name}: {crime_count} crimes",
                fill_opacity=0.7
            ))

    time.sleep(1)  # Respect Overpass API rate limit

# Add marker group and layer control
crime_map.add_child(marker_group)
crime_map.add_child(folium.LayerControl())

# Save and done
crime_map.save("crime_map.html")
print("Map saved to crime_map.html")


Map saved to crime_map.html
